## Generate embeddings with the OpenAI SDK

To run this Notebook, you need to install Polyglot Notebooks for Visual Studio Code, which allows you to run .NET code in Jupyter Notebooks. You can find it here: [Polyglot Notebooks for Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).

### Set up the OpenAI client with GitHub Models

In [ ]:
// Install required NuGet packages for .NET Interactive:
#r "nuget: OpenAI, *-*"
#r "nuget: DotNetEnv, 3.0.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using OpenAI;
using OpenAI.Embeddings;
using DotNetEnv;
using System.ClientModel;


// Load environment variables
Env.Load();

const string MODEL_NAME = "openai/text-embedding-3-small";

var endpoint = "https://models.github.ai/inference";
var credential = System.Environment.GetEnvironmentVariable("GITHUB_TOKEN");

var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri(endpoint)
};

EmbeddingClient client = new(MODEL_NAME, new ApiKeyCredential(credential), openAIOptions);


In [ ]:
string contentInput = "Hoja de vida: Lionel Messi. Futbolista argentino, considerado uno de los mejores jugadores de fútbol de todos los tiempos.";

In [ ]:
public static ReadOnlyMemory<float> PrintAndReturnEmbeddings(OpenAIEmbeddingCollection response)
{
    var embedding = response.FirstOrDefault();
    ReadOnlyMemory<float> vector = embedding.ToFloats();
    int length = vector.Length;
    System.Console.Write($"data[{embedding.Index}]: length={length}, ");
    System.Console.Write($"[{vector.Span[0]}, {vector.Span[1]}, ..., ");
    System.Console.WriteLine($"{vector.Span[length - 2]}, {vector.Span[length - 1]}]");
    return vector;
}

In [ ]:
OpenAIEmbeddingCollection response = client.GenerateEmbeddings(
    new List<string>{
        contentInput
    }
);

var embeddingsResponse1 = PrintAndReturnEmbeddings(response);

### Vector similarity

In [ ]:
public static double CosineSimilarity(float[] v1, float[] v2)
{
    if (v1.Length != v2.Length)
        throw new ArgumentException("Vectors must have the same length");
    
    double dotProduct = v1.Zip(v2, (a, b) => (double)a * b).Sum();
    
    double magnitude1 = Math.Sqrt(v1.Sum(a => (double)a * a));
    double magnitude2 = Math.Sqrt(v2.Sum(a => (double)a * a));
    double magnitude = magnitude1 * magnitude2;
    
    return dotProduct / magnitude;
}

In [ ]:
var response2 = client.GenerateEmbeddings(
    new List<string>{
        "Diego Zumárraga Mera"
    }
);

var embeddingsResponse2 = PrintAndReturnEmbeddings(response2);

In [ ]:
// Compare the two vectors
double similarity = CosineSimilarity(embeddingsResponse1.ToArray(), embeddingsResponse2.ToArray());
Console.WriteLine($"Similarity: {similarity:F4}");

In [ ]:
var response3 = client.GenerateEmbeddings(
    new List<string>{
        "Resume la hoja de vida de Diego Zumárraga Mera"
    }
);

var embeddingsResponse3 = PrintAndReturnEmbeddings(response3);

In [ ]:
// Compare the two vectors
double similarity2 = CosineSimilarity(embeddingsResponse1.ToArray(), embeddingsResponse3.ToArray());
Console.WriteLine($"Similarity: {similarity2:F4}");